# Transformers 模型量化技术：GPTQ

![image](https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/blog/159_autogptq_transformers/thumbnail.jpg)

2022年，Frantar等人发表了论文 [GPTQ：Accurate Post-Training Quantization for Generative Pre-trained Transformers](https://arxiv.org/abs/2210.17323)。

这篇论文详细介绍了一种训练后量化算法，适用于所有通用的预训练 Transformer模型，同时只有微小的性能下降。

GPTQ算法需要通过对量化模型进行推理来校准模型的量化权重。详细的量化算法在[原始论文](https://arxiv.org/pdf/2210.17323.pdf)中有描述。

基于[`auto-gptq`](https://github.com/PanQiWei/AutoGPTQ.git) 开源实现库，transformers 支持使用GPTQ算法量化的模型。

## 使用 GPTQ 量化模型

为了使用 `auto-gptq` 库量化一个模型，我们需要向量化器传递一个数据集。

通常有两种方式构造数据集：
- 量化器支持的默认数据集（包括`['wikitext2','c4','c4-new','ptb','ptb-new']`）
- 一个字符串列表（这些字符串将被用作数据集）

### 使用 GPTQ 算法支持的默认数据集来量化

In [1]:
# 加载数据集

from datasets import load_dataset
wikidata = load_dataset("/data/zlhu/LLM-quickstart/hf/datasets/wikitext/wikitext-2-v1")['test']['text']

在下面的示例中，让我们尝试使用`"wikitext2"`数据集将模型量化为4位精度。支持的精度有`[2, 4, 6, 8]`。

In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer, GPTQConfig
import torch

import os
# 设置镜像源地址
os.environ["HF_ENDPOINT"] = "https://hf-mirror.com"
os.environ['HF_HOME'] = '/data/zlhu/LLM-quickstart/hf'
os.environ['HF_HUB_CACHE'] = '/data/zlhu/LLM-quickstart/hf/hub'

model_name_or_path = "/data/zlhu/LLM-quickstart/hf/hub/opt-2.7b"

quantization_config = GPTQConfig(
     bits=4, # 量化精度
     group_size=128,
     dataset=wikidata,
     desc_act=False,
)

#### 逐层量化

关于 `CUDA extension not installed` 的说明: https://github.com/AutoGPTQ/AutoGPTQ/issues/249

In [3]:
import datasets 

quant_model = AutoModelForCausalLM.from_pretrained(
    model_name_or_path,
    quantization_config=quantization_config,
    device_map='auto')

CUDA extension not installed.
CUDA extension not installed.


Quantizing model.decoder.layers blocks :   0%|          | 0/32 [00:00<?, ?it/s]

Quantizing layers inside the block:   0%|          | 0/6 [00:00<?, ?it/s]

Quantizing layers inside the block:   0%|          | 0/6 [00:00<?, ?it/s]

Quantizing layers inside the block:   0%|          | 0/6 [00:00<?, ?it/s]

...

Quantizing layers inside the block:   0%|          | 0/6 [00:00<?, ?it/s]

Quantizing layers inside the block:   0%|          | 0/6 [00:00<?, ?it/s]

Quantizing layers inside the block:   0%|          | 0/6 [00:00<?, ?it/s]

Quantizing layers inside the block:   0%|          | 0/6 [00:00<?, ?it/s]

Quantizing layers inside the block:   0%|          | 0/6 [00:00<?, ?it/s]

### 实测 GPTQ 量化模型：GPU显存占用峰值超过10GB

```shell
Wed Feb  7 22:03:42 2024
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 545.23.08              Driver Version: 545.23.08    CUDA Version: 12.3     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:0D.0 Off |                    0 |
| N/A   66C    P0              69W /  70W |   9719MiB / 15360MiB |    100%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+----------------------+

+---------------------------------------------------------------------------------------+
| Processes:                                                                            |
|  GPU   GI   CI        PID   Type   Process name                            GPU Memory |
|        ID   ID                                                             Usage      |
|=======================================================================================|
|    0   N/A  N/A      2334      C   /root/miniconda3/bin/python               10714MiB |
+---------------------------------------------------------------------------------------+
```

### 检查量化模型正确性

通过检查`线性层的属性`来确保模型已正确量化，它们应该包含`qweight`和`qzeros`属性，这些属性应该是`torch.int32`数据类型。

In [6]:
quant_model.model.decoder.layers[0].self_attn.q_proj.__dict__

{'training': True,
 '_parameters': OrderedDict(),
 '_buffers': OrderedDict([('qweight',
               tensor([[ 1766754698, -1249142373,  1183631034,  ..., -2038658921,
                        -2037544795, -1956877206],
                       [ 1772644489,  1739893370, -1500087466,  ...,  2021033895,
                         -662329995,  1739241850],
                       [-1164093850, -2007414633, -1951893913,  ..., -1429760649,
                         -979785307,  1451914633],
                       ...,
                       [ 2025363833, -1430615370,  1537972106,  ...,   -56907095,
                        -1737062521,  2092541289],
                       [ 1753842824, -1432974714, -1788312443,  ..., -1160750984,
                        -1983351447,  1449641373],
                       [ 2037938326, -2004444805, -1182099061,  ...,  2000055125,
                         1754823336,  1953973381]], device='cuda:0', dtype=torch.int32)),
              ('qzeros',
               tensor(

In [13]:
base_model = AutoModelForCausalLM.from_pretrained(
    model_name_or_path,
    device_map='auto')

In [14]:
base_model.model.decoder.layers[0].self_attn.q_proj.__dict__

{'training': False,
 '_parameters': OrderedDict([('weight',
               Parameter containing:
               tensor([[ 0.0109,  0.0019,  0.0061,  ..., -0.0097,  0.0081, -0.0143],
                       [ 0.0122,  0.0041,  0.0047,  ..., -0.0040, -0.0068, -0.0022],
                       [ 0.0103,  0.0122, -0.0070,  ..., -0.0005,  0.0052,  0.0169],
                       ...,
                       [-0.0063,  0.0079,  0.0016,  ..., -0.0271, -0.0070, -0.0066],
                       [-0.0230,  0.0130, -0.0002,  ...,  0.0053,  0.0009, -0.0083],
                       [ 0.0117, -0.0104,  0.0215,  ..., -0.0037, -0.0172, -0.0024]],
                      device='cuda:0', requires_grad=True)),
              ('bias',
               Parameter containing:
               tensor([-0.1272,  0.0172,  0.0103,  ..., -0.0928,  0.0567,  0.0510],
                      device='cuda:0', requires_grad=True))]),
 '_buffers': OrderedDict(),
 '_non_persistent_buffers_set': set(),
 '_backward_pre_hooks': Order

In [7]:
# 保存模型权重
quant_model.save_pretrained("models/opt-2.7b-gptq")

#### 使用 GPU 加载模型并生成文本

In [9]:
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path)

text = "Merry Christmas! I'm glad to"
inputs = tokenizer(text, return_tensors="pt").to(0)

out = quant_model.generate(**inputs, max_new_tokens=64)
print(tokenizer.decode(out[0], skip_special_tokens=True))

Merry Christmas! I'm glad to see you're still around.
I'm still around, just not as much as I used to be.


### 使用自定义数据集量化模型（灵活可扩展，前提是准备好数据）

下面演示通过传递自定义数据集来量化一个模型。

通过字符串列表来自定义一个数据集，建议样本数不少于128（样本数太少会影响模型性能）

In [10]:
from transformers import AutoModelForCausalLM, GPTQConfig, AutoTokenizer

model_name_or_path = "facebook/opt-2.7b"
custom_dataset = ["auto-gptq is an easy-to-use model quantization library with user-friendly apis, based on GPTQ algorithm."]

custom_quantization_config = GPTQConfig(
    bits=4,
    group_size=128,
    desc_act=False,
    dataset=custom_dataset
)

custom_quant_model = AutoModelForCausalLM.from_pretrained(model_name_or_path,
                                                          quantization_config=custom_quantization_config,
                                                          torch_dtype=torch.float16,
                                                          device_map="auto")

/data/zlhu/miniconda3/envs/peft/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Quantizing model.decoder.layers blocks :   0%|          | 0/32 [00:00<?, ?it/s]

Quantizing layers inside the block:   0%|          | 0/6 [00:00<?, ?it/s]

Quantizing layers inside the block:   0%|          | 0/6 [00:00<?, ?it/s]

Quantizing layers inside the block:   0%|          | 0/6 [00:00<?, ?it/s]

...

Quantizing layers inside the block:   0%|          | 0/6 [00:00<?, ?it/s]

Quantizing layers inside the block:   0%|          | 0/6 [00:00<?, ?it/s]

Quantizing layers inside the block:   0%|          | 0/6 [00:00<?, ?it/s]

Quantizing layers inside the block:   0%|          | 0/6 [00:00<?, ?it/s]

Quantizing layers inside the block:   0%|          | 0/6 [00:00<?, ?it/s]

相比使用默认数据集，未经精心准备的自定义数据集会明显降低模型性能

In [12]:
text = "Merry Christmas! I'm glad to"
inputs = tokenizer(text, return_tensors="pt").to(0)

out = custom_quant_model.generate(**inputs, max_new_tokens=64)
print(tokenizer.decode(out[0], skip_special_tokens=True))

Merry Christmas! I'm glad to.
.


## [Optional]Homework: 使用 GPTQ 算法量化 OPT-6.7B 模型
